In [ ]:
from agent import Chatbot, Text2SQL
from agent.const import (
    ChatConfig,
    Text2SQLConfig,
    GEMINI_FAST_CONFIG,
    GPT4O_MINI_CONFIG,
    GPT4O_CONFIG,
    TEXT2SQL_FASTEST_CONFIG,
    TEXT2SQL_FAST_OPENAI_CONFIG,
    TEXT2SQL_DEEPSEEK_V3_CONFIG,
    TEXT2SQL_DEEPSEEK_V3_FAST_CONFIG
)

from agent.prompt.prompt_controller import (
    PromptConfig, 
    VERTICAL_PROMPT_BASE, 
    VERTICAL_PROMPT_UNIVERSAL,
    HORIZONTAL_PROMPT_BASE,
    HORIZONTAL_PROMPT_UNIVERSAL,
    FIIN_VERTICAL_PROMPT_UNIVERSAL,
)

from ETL.dbmanager.setup import (
    DBConfig,
    BGE_VERTICAL_BASE_CONFIG,
    BGE_VERTICAL_UNIVERSAL_CONFIG,
    BGE_HORIZONTAL_BASE_CONFIG,
    TEI_VERTICAL_UNIVERSAL_CONFIG,
    OPENAI_VERTICAL_UNIVERSAL_CONFIG,
    setup_db
)

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
import os

import logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

from ETL.connector import check_embedding_server
from ETL.dbmanager import get_semantic_layer, BaseRerannk

In [ ]:
chat_config = ChatConfig(**GPT4O_MINI_CONFIG)
text2sql_config = Text2SQLConfig(**TEXT2SQL_DEEPSEEK_V3_FAST_CONFIG)
prompt_config = PromptConfig(**FIIN_VERTICAL_PROMPT_UNIVERSAL)

embedding_server = os.getenv('EMBEDDING_SERVER_URL')

if check_embedding_server(embedding_server):
    logging.info('Using remote embedding server')
    db_config = DBConfig(**TEI_VERTICAL_UNIVERSAL_CONFIG)
elif os.path.exists('data/vector_db_vertical_openai'):
    logging.info('Using openai embedding')
    db_config = DBConfig(**OPENAI_VERTICAL_UNIVERSAL_CONFIG)

elif os.getenv('LOCAL_EMBEDDING'):
    import torch

    db_config = DBConfig(**BGE_VERTICAL_UNIVERSAL_CONFIG)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')    
    embedding_model = HuggingFaceEmbeddings(model_name='BAAI/bge-base-en-v1.5', model_kwargs = {'device': device})
    db_config.embedding = embedding_model

else:
    raise ValueError('No Embedding Method Found')


logging.info('Finish setup embedding')

reranker = BaseRerannk(name=os.getenv('RERANKER_SERVER_URL'))

logging.info(f'Finish setup reranker, using reranker {reranker.reranker_type}')
